# Polarsによる時系列データの取り扱い
面倒な時系列データをPolarsで取り扱う方法をメモしておく。  
必要であればPandas DataFrameも使う。

In [2]:
import polars as pl
import pandas as pd
import seaborn as sns
import numpy as np
import os
import math
import pyarrow as pa
import pyarrow.parquet as pq
import itertools
from pathlib import Path
from glob import glob
from sklearn.preprocessing import StandardScaler, LabelEncoder

%matplotlib inline
import matplotlib.pyplot as plt

## 様々な形式で表現された時刻データの読み込み方
ログの分析などではナノ秒まで情報が必要な場合があるので、  
適宜ナノ秒まで含めて取り扱う。

### - 時刻データが綺麗にフォーマットされている場合

In [3]:
df_1 = pl.DataFrame({'Time': 
         ['2020-01-01 00:00:00.123456789',
          '2020-01-01 00:00:00.123456790',
          '2020-01-01 00:00:00.123456791']
        })

In [4]:
# カラムの横幅を40文字まで拡張して表示
pl.Config.set_fmt_str_lengths(40)
df_1

Time
str
"""2020-01-01 00:00:00.123456789"""
"""2020-01-01 00:00:00.123456790"""
"""2020-01-01 00:00:00.123456791"""


pl.DataFrameのschemaにpl.Datetime("ns")を指定するのはダメそうだったので、  
strで一旦読み込む。  
カラムの幅が狭くてナノ秒まで見えないときはpl.Config.set_fmt_str_lengths(文字数)  で調整するとよい。  
https://pola-rs.github.io/polars/py-polars/html/reference/api/polars.Config.set_fmt_str_lengths.html

In [28]:
df_1.get_column('Time').str.strptime(pl.Datetime("ns"))

Time
datetime[ns]
2020-01-01 00:00:00.123456789
2020-01-01 00:00:00.123456790
2020-01-01 00:00:00.123456791


ナノ秒まで含めてDatetime型に変換するには上記のように書けばよい。  
https://stackoverflow.com/questions/75119895/parse-nanoseconds-with-polars

### - 数字の羅列になっている場合

In [29]:
df_2 = pl.DataFrame({'Time': 
         ['20200101000000123456789',
          '20200101000000123456790',
          '20200101000000123456791']
        })

In [30]:
df_2

Time
str
"""20200101000000123456789"""
"""20200101000000123456790"""
"""20200101000000123456791"""


In [31]:
# ナノ秒まで含めてDatetime型に変換
df_2.get_column('Time').str.strptime(pl.Datetime('ns'), '%Y%m%d%H%M%S%f')

Time
datetime[ns]
2020-01-01 00:00:00.123456789
2020-01-01 00:00:00.123456790
2020-01-01 00:00:00.123456791


上記のように書くことで、数字の羅列もDatetime型へ変換可能。  
フォーマットの指定には'-'や':'は不要っぽい。  
pl.Datetime('ms')などにすることで秒以下の単位を変更できる。

In [32]:
df_2.get_column('Time').str.strptime(pl.Datetime('ns'), '%Y%m%d%H%M%S%f').diff()

Time
duration[ns]
null
1ns
1ns


.diff()でひとつ前のレコードとの時刻差を算出できる。

### - 年月日と時刻が別のカラムの場合

In [9]:
df_3 = pl.DataFrame({
    'Date':['20200101',
            '20200102',
            '20200103'],
    'Time': ['131205123456789',
             '133406123456790',
             '142215123456791']
        })

In [7]:
df_3

Date,Time
str,str
"""20200101""","""131205123456789"""
"""20200102""","""133406123456790"""
"""20200103""","""142215123456791"""


In [8]:
df_3.select(
    (pl.col('Date') + pl.col('Time')).str
    .strptime(pl.Datetime('ns'), '%Y%m%d%H%M%S%f').alias('Date_and_Time')
)

Date_and_Time
datetime[ns]
2020-01-01 13:12:05.123456789
2020-01-02 13:34:06.123456790
2020-01-03 14:22:15.123456791
